# Práctica 3: Representación del conocimiento

__Fecha de entrega: 8 de mayo de 2022__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 2__

__Nombres de los estudiantes: Enrique Rey Gisbert, Juan Carlos Llamas Núñez y Alberto Almagro Sánchez__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [2]:
import sys
!{sys.executable} -m pip install wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [69]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
    ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[Isaac Asimov (Q34981)](https://www.wikidata.org/wiki/Q34981) fue un escritor y profesor de bioquímica en la Universidad de Boston conocido por ser un prolífico autor de obras de ciencia ficción, historia y divulgación científica. 

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

In [70]:
%%wdsparql
SELECT ?fecha ?fechaLabel ?lugar ?lugarLabel
WHERE 
{
    wd:Q34981 wdt:P569 ?fecha.   # Isaac Asimov Q34981, Fecha de nacimiento P569
    wd:Q34981 wdt:P19 ?lugar.    # Isaac Asimov Q34981, Lugar de nacimiento P19
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,fecha,fechaLabel,lugar,lugarLabel
0,1920-01-02T00:00:00Z,1920-01-02T00:00:00Z,http://www.wikidata.org/entity/Q1010621,Petrovichi


### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

In [71]:
%%wdsparql
SELECT ?prof ?profLabel
WHERE {
    wd:Q34981 wdt:P106 ?prof.     # Isaac Asimov Q34981, Profesiones P106
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?profLabel)        # Ordenados alfabeticamente por nombre de la profesión

,prof,profLabel
0,http://www.wikidata.org/entity/Q18814623,autobiographer
1,http://www.wikidata.org/entity/Q2919046,biochemist
2,http://www.wikidata.org/entity/Q1930187,journalist
3,http://www.wikidata.org/entity/Q15980158,non-fiction writer
4,http://www.wikidata.org/entity/Q6625963,novelist
5,http://www.wikidata.org/entity/Q12144794,prosaist
6,http://www.wikidata.org/entity/Q18844224,science fiction writer
7,http://www.wikidata.org/entity/Q3745071,science writer
8,http://www.wikidata.org/entity/Q901,scientist
9,http://www.wikidata.org/entity/Q28389,screenwriter


### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

Planteamos dos posibles consultas según lo que realmente se desee obtener. Si únicamente se desea obtener las profesiones que sonb subtipo directo de EScritor, usaremos la primera consulta. Si queremos obtener las profesiones que son subtipo directo o indirecto (vease el cierre transitivo de la relación subtipo), usaremos la segunda. Entendemos que la que se pide es la segunda.

In [72]:
%%wdsparql
SELECT ?prof ?profLabel
WHERE {
    wd:Q34981 wdt:P106 ?prof.     # Isaac Asimov Q34981, Profesiones P106
    ?prof wdt:P279 wd:Q36180.     # Profesiones que son subtipo (P279) de Escritor Q36180 (sin cierre transitivo, solo subtipo directo)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?profLabel)          # Ordenados alfabeticamente por nombre de la profesión

,prof,profLabel
0,http://www.wikidata.org/entity/Q1930187,journalist
1,http://www.wikidata.org/entity/Q15980158,non-fiction writer
2,http://www.wikidata.org/entity/Q6625963,novelist
3,http://www.wikidata.org/entity/Q12144794,prosaist
4,http://www.wikidata.org/entity/Q3745071,science writer
5,http://www.wikidata.org/entity/Q28389,screenwriter


In [73]:
%%wdsparql
SELECT ?prof ?profLabel
WHERE {
    wd:Q34981 wdt:P106 ?prof.     # Isaac Asimov Q34981, Profesiones P106
    ?prof wdt:P279* wd:Q36180.    # Profesiones que son subtipo (P279) de Escritor Q36180 (con cierre transitivo)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?profLabel)          # Ordenados alfabeticamente por nombre de la profesión

,prof,profLabel
0,http://www.wikidata.org/entity/Q18814623,autobiographer
1,http://www.wikidata.org/entity/Q2919046,biochemist
2,http://www.wikidata.org/entity/Q1930187,journalist
3,http://www.wikidata.org/entity/Q15980158,non-fiction writer
4,http://www.wikidata.org/entity/Q6625963,novelist
5,http://www.wikidata.org/entity/Q12144794,prosaist
6,http://www.wikidata.org/entity/Q18844224,science fiction writer
7,http://www.wikidata.org/entity/Q3745071,science writer
8,http://www.wikidata.org/entity/Q901,scientist
9,http://www.wikidata.org/entity/Q28389,screenwriter


Observamos que en la segunda consultas se obtienen, además de todos los resultados de la primer, el resto de profesiones. Es decir, todas las profesiones de Asimov son subclase (indirectamente) de Escritor. Por ejemplo, aunque pudiera parecer que biochemist no es subclase de escritor, se tiene que:


    bichemist subclass of interdisciplinary scientist subclass of scientist subclass of researcher subclass of non-fiction writer subclass of writer. 
    
    
Intuitivamente, parece que la segunda consulta ha recuperado más profesiones de las deseasdas.

### Consulta 4

Asimov se casó más de una vez. Para cada uno de los matrimonios queremos conocer el nombre de la esposa y las fechas de inicio y finalización. Los resultados deben aparecer ordenados cronológicamente.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [74]:
%%wdsparql
SELECT ?esposa ?esposaLabel ?inicio ?inicioLabel ?fin ?finLabel
WHERE {
    wd:Q34981 p:P26 ?nodo.    # Isaac Asimov Q34981, Nodo Statement de Spouse P26
    ?nodo ps:P26 ?esposa.     # Nodo Statement de Spouse (?nodo) a Objeto del nodo (ps:P26)
    ?nodo pq:P580 ?inicio.    # Nodo Statement de Spouse (?nodo) a Cualificador start time (pq:P580)
    ?nodo pq:P582 ?fin.       # Nodo Statement de Spouse (?nodo) a Cualificador end time (pq:P582)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?inicioLabel)    # Ordenados cronologicamente por inicio del matrimonio

,esposa,esposaLabel,inicio,inicioLabel,fin,finLabel
0,http://www.wikidata.org/entity/Q107693884,Gertrude Asimov,1942-07-26T00:00:00Z,1942-07-26T00:00:00Z,1973-01-01T00:00:00Z,1973-01-01T00:00:00Z
1,http://www.wikidata.org/entity/Q517435,Janet Asimov,1973-01-01T00:00:00Z,1973-01-01T00:00:00Z,1992-01-01T00:00:00Z,1992-01-01T00:00:00Z


### Consulta 5

Asimov recibió muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha y la obra por la que fue premiado (si están disponibles). Los resultados se deben mostrar ordenador por fecha.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [75]:
%%wdsparql
SELECT ?premio ?premioLabel ?fecha ?fechaLabel ?obra ?obraLabel
WHERE {
    wd:Q34981 p:P166 ?nodo.                # Isaac Asimov Q34981, nodo Statement awards received P166
    ?nodo ps:P166 ?premio.                 # nodo Statement de awards received (?nodo) a objeto Premios (ps:P166)
    OPTIONAL { ?nodo pq:P1686 ?obra. }     #  nodo Statement de awards received (?nodo) a Cualificador for work (pq:1686)
    OPTIONAL { ?nodo pq:P585 ?fecha. }     #  nodo Statement de awards received (?nodo) a Cualificador point in time (pq:585)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?fechaLabel)                  # Ordenados cronologicamente por fecha

,premio,premioLabel,fecha,fechaLabel,obra,obraLabel
0,http://www.wikidata.org/entity/Q61745062,Fellow of the Committee for Skeptical Inquiry,None,None,None,None
1,http://www.wikidata.org/entity/Q52382875,AAAS Fellow,None,None,None,None
2,http://www.wikidata.org/entity/Q188914,Hugo Award,1963-01-01T00:00:00Z,1963-01-01T00:00:00Z,None,None
3,http://www.wikidata.org/entity/Q6143927,James T. Grady-James H. Stack Award for Interp...,1965-01-01T00:00:00Z,1965-01-01T00:00:00Z,None,None
4,http://www.wikidata.org/entity/Q188914,Hugo Award,1966-01-01T00:00:00Z,1966-01-01T00:00:00Z,http://www.wikidata.org/entity/Q20899148,Foundation Trilogy
5,http://www.wikidata.org/entity/Q987744,Edward E. Smith Memorial Award,1967-01-01T00:00:00Z,1967-01-01T00:00:00Z,None,None
6,http://www.wikidata.org/entity/Q266012,Nebula Award for Best Novel,1972-01-01T00:00:00Z,1972-01-01T00:00:00Z,http://www.wikidata.org/entity/Q389750,The Gods Themselves
7,http://www.wikidata.org/entity/Q255032,Hugo Award for Best Novel,1973-01-01T00:00:00Z,1973-01-01T00:00:00Z,http://www.wikidata.org/entity/Q389750,The Gods Themselves
8,http://www.wikidata.org/entity/Q595998,Locus Award for Best Novel,1973-01-01T00:00:00Z,1973-01-01T00:00:00Z,http://www.wikidata.org/entity/Q389750,The Gods Themselves
9,http://www.wikidata.org/entity/Q3174253,Klumpke-Roberts Award,1975-01-01T00:00:00Z,1975-01-01T00:00:00Z,None,None


### Consulta 6

Ahora queremos conocer todas las obras escritas por Asimov del tipo [obras literarias de ciencia ficción (Q3238422)](https://www.wikidata.org/wiki/Q3238422) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

En primer lugar, notemos que si simplemente ejecutamos la siguiente consulta:

In [11]:
%%wdsparql
SELECT DISTINCT ?genero ?generoLabel
WHERE {
    ?genero wdt:P279 wd:Q3238422.    # Subgéneros (P279) de literatura de ciencia ficción (Q3238422)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?titulo)               # Ordenados alfabeticamente por nombre

,genero,generoLabel
0,http://www.wikidata.org/entity/Q12132683,science fiction novel
1,http://www.wikidata.org/entity/Q20443047,dystopian literature
2,http://www.wikidata.org/entity/Q21905924,science fiction short story
3,http://www.wikidata.org/entity/Q67467614,merveilleux scientifique


No aparece 'science fiction literature como género. Por tanto, lo añadimos manualmente como en la siguiente consulta.

In [12]:
%%wdsparql
SELECT DISTINCT ?genero ?generoLabel
WHERE {
    {?genero wdt:P279 wd:Q3238422.}    # Subgéneros (P279) de literatura de ciencia ficción (Q3238422)
    UNION {BIND(wd:Q3238422 as ?genero).} # Añadimos literatura de ciencia ficción (Q3238422) a los géneros (?genero).
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?titulo)               # Ordenados alfabeticamente por nombre

,genero,generoLabel
0,http://www.wikidata.org/entity/Q12132683,science fiction novel
1,http://www.wikidata.org/entity/Q20443047,dystopian literature
2,http://www.wikidata.org/entity/Q21905924,science fiction short story
3,http://www.wikidata.org/entity/Q67467614,merveilleux scientifique
4,http://www.wikidata.org/entity/Q3238422,science fiction literature


Y la consulta pedida nos resulta:

In [15]:
%%wdsparql
SELECT DISTINCT ?obra ?titulo ?tituloLabel ?genero ?generoLabel
WHERE {
    ?obra wdt:P50 wd:Q34981.         # Obra cuyo autor (P50) es Isaac Asimov Q34981
    ?obra p:P1476 ?nodo.             # Obra (?obra) nodo statement de titulo P1476 (?nodo)
    ?nodo ps:P1476 ?titulo.          # nodo statement de titulo (?nodo) a objeto titulo (ps:P1476)
    {?genero wdt:P279 wd:Q3238422.}    # Subgéneros (P279) de literatura de ciencia ficción (Q3238422)
    UNION {BIND(wd:Q3238422 as ?genero).} # Añadimos literatura de ciencia ficción (Q3238422) a los géneros (?genero).
    ?obra wdt:P136 ?genero.          # Obra (?obra) cuyo género (P136) es ?genero
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?titulo)               # Ordenados alfabeticamente por nombre

,obra,titulo,tituloLabel,genero,generoLabel
0,http://www.wikidata.org/entity/Q4631777,2430 n. Chr.,2430 n. Chr.,http://www.wikidata.org/entity/Q21905924,science fiction short story
1,http://www.wikidata.org/entity/Q5003079,Aus Alt mach Neu,Aus Alt mach Neu,http://www.wikidata.org/entity/Q21905924,science fiction short story
2,http://www.wikidata.org/entity/Q3055501,Das geeignete Studium,Das geeignete Studium,http://www.wikidata.org/entity/Q21905924,science fiction short story
3,http://www.wikidata.org/entity/Q3524680,Der endochronische Effekt,Der endochronische Effekt,http://www.wikidata.org/entity/Q21905924,science fiction short story
4,http://www.wikidata.org/entity/Q7738024,Der größte Aktivposten,Der größte Aktivposten,http://www.wikidata.org/entity/Q21905924,science fiction short story
5,http://www.wikidata.org/entity/Q7677898,Der rettende Gedanke,Der rettende Gedanke,http://www.wikidata.org/entity/Q21905924,science fiction short story
6,http://www.wikidata.org/entity/Q7756390,Die Pause,Die Pause,http://www.wikidata.org/entity/Q21905924,science fiction short story
7,http://www.wikidata.org/entity/Q783533,Die Rettung des Imperiums,Die Rettung des Imperiums,http://www.wikidata.org/entity/Q12132683,science fiction novel
8,http://www.wikidata.org/entity/Q30124284,Die Rettung des Imperiums,Die Rettung des Imperiums,http://www.wikidata.org/entity/Q12132683,science fiction novel
9,http://www.wikidata.org/entity/Q3549799,Ein Denkmal für Vater,Ein Denkmal für Vater,http://www.wikidata.org/entity/Q21905924,science fiction short story


Observamos en la anterior consulta que hay obras con el mismo título que tienen distintos identificadores, como por ejemplo:

    7	http://www.wikidata.org/entity/Q783533	Die Rettung des Imperiums
    8	http://www.wikidata.org/entity/Q30124284	Die Rettung des Imperiums
    
Al investigar un poco más este caso en concreto para comprender porqué ocurre esto, si pinchamos en la referencia 7 nos lleva a un libro titulado 'Prelude to Foundation' de Isaac Asimov que tiene una propiedad 'has edition or translation' con valor 'Die Rettung des Imperiums', pero en la propiedad 'titulo' tiene también el título 'Die Rettung des Imperiums', que es el que estamos obteniendo puesto que 'Prelude to Foundation' es meramente una transcripción.

Nota: En realidad, si omitimos el añadir 'science fiction literature' a los géneros, obtendríamos el mismo resultado, pero hemos preferido añadirlo porque creemos que se ajusta más al resultado.

### Consulta 7

Vamos a volver a recuperar las mismas obras de la consulta anterior pero mostrando también la fecha de publicación de cada una. Las obras deben aparece aunque no tengan fecha de publicación asociada. En caso de que una obra tenga varias fechas de publicación, se mostrará sólo la más antigua. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por obra y aplicar una función de agregación sobre las fechas de publicación. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

In [77]:
%%wdsparql
SELECT DISTINCT ?obra ?titulo ?tituloLabel (MIN(?fecha) AS ?minFecha)     # Función de agregación MIN
WHERE {
    ?obra wdt:P50 wd:Q34981.         # Obra cuyo autor (P50) es Isaac Asimov Q34981
    ?obra p:P1476 ?nodo.             # Obra (?obra) nodo statement de titulo P1476 (?nodo)
    ?nodo ps:P1476 ?titulo.         # nodo statement de titulo (?nodo) a titulo (ps:P1476)
    ?genero wdt:P279 wd:Q3238422.   # Subtipos (P279) de literatura de ciencia ficción Q3238422
    ?obra wdt:P136 ?genero.          # Obra (?obra) cuyo género (P136) es ?genero
    OPTIONAL { ?obra wdt:P577 ?fecha }          # Obra (?obra) cuya fecha de publicación (P577) es ?fecha
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?obra ?titulo ?tituloLabel    # Agrupados por obra
ORDER BY ASC(?minFecha)               # Ordenados por orden cronológico

,obra,titulo,tituloLabel,minFecha
0,http://www.wikidata.org/entity/Q716005,Satisfaction Guaranteed,Satisfaction Guaranteed,None
1,http://www.wikidata.org/entity/Q3873373,Marooned off Vesta,Marooned off Vesta,1939-03-01T00:00:00Z
2,http://www.wikidata.org/entity/Q7721040,The Callistan Menace,The Callistan Menace,1940-01-01T00:00:00Z
3,http://www.wikidata.org/entity/Q5890989,Homo Sol,Homo Sol,1940-09-01T00:00:00Z
4,http://www.wikidata.org/entity/Q372968,Robbie,Robbie,1940-09-01T00:00:00Z
5,http://www.wikidata.org/entity/Q1138091,Nightfall,Nightfall,1941-01-01T00:00:00Z
6,http://www.wikidata.org/entity/Q737630,Reason,Reason,1941-04-01T00:00:00Z
7,http://www.wikidata.org/entity/Q3345927,Christmas on Ganymede,Christmas on Ganymede,1942-01-01T00:00:00Z
8,http://www.wikidata.org/entity/Q555208,Victory Unintentional,Victory Unintentional,1942-01-01T00:00:00Z
9,http://www.wikidata.org/entity/Q785983,The Hazing,The Hazing,1942-01-01T00:00:00Z


Miremos ahora, para comprobar que la anterior consulta hace lo que tiene que hacer, cuántas fechas de publicación tienen estas obras. Para ello, (en vez de utilizar la función de agregación MIN) vamos a utilizar COUNT sobre las fechas de publicación.

In [78]:
%%wdsparql
SELECT DISTINCT ?obra ?titulo ?tituloLabel (COUNT(?fecha) AS ?countFecha)     # Función de agregación COUNT
WHERE {
    ?obra wdt:P50 wd:Q34981.         # Obra cuyo autor (P50) es Isaac Asimov Q34981
    ?obra p:P1476 ?nodo.             # Obra (?obra) nodo statement de titulo P1476 (?nodo)
    ?nodo ps:P1476 ?titulo.          # nodo statement de titulo (?nodo) a titulo (ps:P1476)
    ?genero wdt:P279 wd:Q3238422.    # Subtipos (P279) de literatura de ciencia ficción Q3238422
    ?obra wdt:P136 ?genero.          # Obra (?obra) cuyo género (P136) es ?genero
    OPTIONAL { ?obra wdt:P577 ?fecha }          # Obra (?obra) cuya fecha de publicación (P577) es ?fecha
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?obra ?titulo ?tituloLabel     # Agrupados por obra
ORDER BY ASC(?countFecha)               # Ordenados por orden cronológico

,obra,titulo,tituloLabel,countFecha
0,http://www.wikidata.org/entity/Q716005,Satisfaction Guaranteed,Satisfaction Guaranteed,0
1,http://www.wikidata.org/entity/Q2079836,Franchise,Franchise,1
2,http://www.wikidata.org/entity/Q2647714,Spell My Name with an S,Spell My Name with an S,1
3,http://www.wikidata.org/entity/Q2855170,Breeds There a Man...?,Breeds There a Man...?,1
4,http://www.wikidata.org/entity/Q2864287,Death Sentence,Death Sentence,1
5,http://www.wikidata.org/entity/Q3055501,The Proper Study,The Proper Study,1
6,http://www.wikidata.org/entity/Q3055501,Das geeignete Studium,Das geeignete Studium,1
7,http://www.wikidata.org/entity/Q3061953,Verbannungsort Hölle,Verbannungsort Hölle,1
8,http://www.wikidata.org/entity/Q3061953,Exile to Hell,Exile to Hell,1
9,http://www.wikidata.org/entity/Q3153721,Insert Knob A in Hole B,Insert Knob A in Hole B,1


Como era de esperar, la mayoría de obras tienen 1 única fecha de publicación, pero hay una obra titulada 'Satisfaction Guaranteed' sin fecha de publicación (que estamos teniendo en cuenta usando el OPTIONAL en la consulta original) y algunas obras con 2 fechas de publicación (una de ellas es nuestra ya conocida 'Die Rettung des Imperiums'), para las cuales efectivamente estamos obteniendo la más antigua utilizando MIN.

### Consulta 8

Asimov es sin duda un autor prolífico. ¿Cuántas obras escribió a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

Procedemos de manera similar al apartado anterior: buscamos las obras cuyo autor es Isaac Asimov y sus títulos. Además, nos quedamos con las obras que sean instancias de la clase obra literaria. Para contar el total de obras en estas condiciones debemos usar la funcion de agregación COUNT, pero debemos agrupar sobre un atributo común a todos los resultados. Para crear dicho atributo común de manera artificial, introducimos la variable ?aux que será la instancia de la obra. Ingenuamente podríamos pensar que esta debe ser obra literaria porque hemos impuesto esa condición, pero como una obra puede ser instancia de varias clases distintas, filtramos para que dicha clase sea obligatoriamente obra literaria. Después, simplemente aplicamos la función de agrecación sobre ?aux.

In [80]:
%%wdsparql
SELECT (COUNT(?aux) AS ?NumObras)
WHERE {
    ?obra wdt:P50 wd:Q34981.         # Obra cuyo autor (P50) es Isaac Asimov Q34981
    ?obra p:P1476 ?nodo.             # Obra (?obra) nodo statement de titulo P1476 (?nodo)
    ?nodo ps:P1476 ?titulo.          # nodo statement de titulo (?nodo) a titulo (ps:P1476)
    ?obra wdt:P31 wd:Q7725634.       # Obra (?obra) instancia de (P31) obra litararia (Q7725634)
    ?obra wdt:P31 ?aux.              # Obra (?obra) instancia de (P31) ?aux
    FILTER(?aux=wd:Q7725634).        # Imponemos que ?aux sea obra litararia (Q7725634)
    #Así todos los resultados recuperados tienen una columna común aux sobre la que agrupar
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?aux 

,NumObras
0,212


### Consulta 9

De todas las obras que escribió a lo largo de su vida, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

Esta primera consulta recupera los títulos de las obras y sus fechas de publicación siempre que estas estén entre 1970 y 1980. Cuando una obra tiene varias publicaciones en este intervalo de tiempo, aparecen ambas publicaciones. Por ejemplo, la obra 'Have You Seen These?' aparece 2 veces porque tiene dos publicaciones, una en enero de 1974 y otra en marzo de ese mismo año.

In [81]:
%%wdsparql
SELECT ?obra ?titulo ?tituloLabel ?fecha ?fechaLabel
WHERE {
    ?obra wdt:P50 wd:Q34981.          # Obra cuyo autor (P50) es Isaac Asimov Q34981
    ?obra p:P1476 ?nodo.              # Obra (?obra) nodo statement de titulo P1476 (?nodo)
    ?nodo ps:P1476 ?titulo.           # nodo statement de titulo (?nodo) a titulo (ps:P1476)
    ?obra wdt:P577 ?fecha             # nodo statement de titulo (?nodo) a publication date (ps:P577)
    FILTER(YEAR(?fecha) <= 1980 && YEAR(?fecha) >= 1970).        # Imponemos que la fecha esté entre 1970 y 1980
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY (?fecha)

,obra,titulo,tituloLabel,fecha,fechaLabel
0,http://www.wikidata.org/entity/Q4631777,2430 n. Chr.,2430 n. Chr.,1970-01-01T00:00:00Z,1970-01-01T00:00:00Z
1,http://www.wikidata.org/entity/Q4631777,2430 A. D.,2430 A. D.,1970-01-01T00:00:00Z,1970-01-01T00:00:00Z
2,http://www.wikidata.org/entity/Q4806996,Asimov's Guide to Shakespeare,Asimov's Guide to Shakespeare,1970-01-01T00:00:00Z,1970-01-01T00:00:00Z
3,http://www.wikidata.org/entity/Q7765145,The Solar System and Back,The Solar System and Back,1970-01-19T00:00:00Z,1970-01-19T00:00:00Z
4,http://www.wikidata.org/entity/Q28808642,As Chemist to Chemist,As Chemist to Chemist,1970-05-01T00:00:00Z,1970-05-01T00:00:00Z
5,http://www.wikidata.org/entity/Q28808642,A Problem of Numbers,A Problem of Numbers,1970-05-01T00:00:00Z,1970-05-01T00:00:00Z
6,http://www.wikidata.org/entity/Q16932955,The Stars in their Courses,The Stars in their Courses,1971-01-01T00:00:00Z,1971-01-01T00:00:00Z
7,http://www.wikidata.org/entity/Q2277195,The Early Asimov,The Early Asimov,1972-01-01T00:00:00Z,1972-01-01T00:00:00Z
8,http://www.wikidata.org/entity/Q831556,Mirror Image,Mirror Image,1972-01-01T00:00:00Z,1972-01-01T00:00:00Z
9,http://www.wikidata.org/entity/Q389750,The Gods Themselves,The Gods Themselves,1972-01-01T00:00:00Z,1972-01-01T00:00:00Z


Alternativamente, si para cada obra solo queremos recuperar una de las fechas de publicación, la primera en el periodo de 1970 a 1980 por ejemplo, haremos la siguiente consulta:

In [82]:
%%wdsparql
SELECT ?titulo ?tituloLabel (MIN(?fecha) AS ?minFecha)
WHERE {
    ?obra wdt:P50 wd:Q34981.          # Obra cuyo autor (P50) es Isaac Asimov Q34981
    ?obra p:P1476 ?nodo.              # Obra (?obra) nodo statement de titulo P1476 (?nodo)
    ?nodo ps:P1476 ?titulo.           # nodo statement de titulo (?nodo) a titulo (ps:P1476)
    ?obra wdt:P577 ?fecha             # nodo statement de titulo (?nodo) a publication date (ps:P577)
    FILTER(YEAR(?fecha) <= 1980 && YEAR(?fecha) >= 1970).        # Imponemos que la fecha esté entre 1970 y 1980
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?titulo ?tituloLabel
ORDER BY (?minFecha)

,titulo,tituloLabel,minFecha
0,2430 n. Chr.,2430 n. Chr.,1970-01-01T00:00:00Z
1,2430 A. D.,2430 A. D.,1970-01-01T00:00:00Z
2,Asimov's Guide to Shakespeare,Asimov's Guide to Shakespeare,1970-01-01T00:00:00Z
3,The Solar System and Back,The Solar System and Back,1970-01-19T00:00:00Z
4,As Chemist to Chemist,As Chemist to Chemist,1970-05-01T00:00:00Z
5,A Problem of Numbers,A Problem of Numbers,1970-05-01T00:00:00Z
6,The Stars in their Courses,The Stars in their Courses,1971-01-01T00:00:00Z
7,The Early Asimov,The Early Asimov,1972-01-01T00:00:00Z
8,Mirror Image,Mirror Image,1972-01-01T00:00:00Z
9,The Gods Themselves,The Gods Themselves,1972-01-01T00:00:00Z


Observamos que la obra 'Have You Seen These?', ya solo aparece una vez y su fecha de publicación es la más temprana (enero de 1974)

### Consulta 10

¿Qué otros escritores de ciencia ficción estudiaron en algún centro donde también estudió Asimov? Para cada uno de ellos muestra su nombre y fechas de nacimiento y defunción (si están disponibles) y el centro donde estudió. Muestra los resultados ordenados alfabéticamente por centro y luego por autor.

In [16]:
%%wdsparql
SELECT DISTINCT ?autor ?autorLabel ?centro ?centroLabel ?nacimiento ?nacimientoLabel ?muerte ?muerteLabel
WHERE {
    wd:Q34981 wdt:P69 ?centro.     # Obtener centros (?centro) donde estudió (wdt:P69) Asimov (wd:Q34981).
    {?genero wdt:P279 wd:Q3238422.}    # Obtener subgéneros (P279) de literatura de ciencia ficción (Q3238422)
    UNION {BIND(wd:Q3238422 as ?genero).} # Añadir literatura de ciencia ficción (Q3238422) a los géneros (?genero).
    ?obra wdt:P136 ?genero.        # Obtener obras (?obra) cuyo género (wdt:P136) es uno de los anteriores (?genero).
    ?obra wdt:P50 ?autor.          # Obtener autores (?autor) que escribieron (wdt:P50) las obras anteriores (?obra).
    ?autor wdt:P69 ?centro.        # Filtrar los autores (?autor) que estudiaron (wdt:P69) en uno de los centros (?centro).
    OPTIONAL {?autor wdt:P569 ?nacimiento}. # Obtener fecha de nacimiento (wdt:P569) de los autores anteriores (?autor).
    OPTIONAL {?autor wdt:P570 ?muerte}.     # Obtener fecha de defunción (wdt:P570) de los autores anteriores (?autor).
    FILTER (?autor != wd:Q34981).           # Filtrar los autores (?autor) para que sean diferentes de Asimov (wd:Q34981).
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?centroLabel ?autorLabel        # Ordenados alfabeticamente por centro y después por nombre.

,autor,autorLabel,centro,centroLabel,nacimiento,nacimientoLabel,muerte,muerteLabel
0,http://www.wikidata.org/entity/Q547414,Algis Budrys,http://www.wikidata.org/entity/Q49088,Columbia University,1931-01-09T00:00:00Z,1931-01-09T00:00:00Z,2008-06-09T00:00:00Z,2008-06-09T00:00:00Z
1,http://www.wikidata.org/entity/Q191716,Gene Roddenberry,http://www.wikidata.org/entity/Q49088,Columbia University,1921-08-19T00:00:00Z,1921-08-19T00:00:00Z,1991-10-24T00:00:00Z,1991-10-24T00:00:00Z
2,http://www.wikidata.org/entity/Q248867,James Blish,http://www.wikidata.org/entity/Q49088,Columbia University,1921-05-23T00:00:00Z,1921-05-23T00:00:00Z,1975-07-30T00:00:00Z,1975-07-30T00:00:00Z
3,http://www.wikidata.org/entity/Q214642,Paul Auster,http://www.wikidata.org/entity/Q49088,Columbia University,1947-02-03T00:00:00Z,1947-02-03T00:00:00Z,None,None
4,http://www.wikidata.org/entity/Q314553,Robert Silverberg,http://www.wikidata.org/entity/Q49088,Columbia University,1935-01-15T00:00:00Z,1935-01-15T00:00:00Z,None,None
5,http://www.wikidata.org/entity/Q295406,Roger Zelazny,http://www.wikidata.org/entity/Q49088,Columbia University,1937-05-13T00:00:00Z,1937-05-13T00:00:00Z,1995-06-14T00:00:00Z,1995-06-14T00:00:00Z
6,http://www.wikidata.org/entity/Q181659,Ursula K. Le Guin,http://www.wikidata.org/entity/Q49088,Columbia University,1929-10-21T00:00:00Z,1929-10-21T00:00:00Z,2018-01-22T00:00:00Z,2018-01-22T00:00:00Z


Resulta interesante ver que todos los escritores estudiaron en la Universidad de Columbia, no habiendo ninguno de los otros centros donde estudió Asimov, que como vemos en la siguiente consulta son 3 más.

In [84]:
%%wdsparql
SELECT DISTINCT ?centro ?centroLabel
WHERE {
    wd:Q34981 wdt:P69 ?centro.     # Obtener centros (?centro) donde estudió (wdt:P69) Asimov (wd:Q34981).
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,centro,centroLabel
0,http://www.wikidata.org/entity/Q5149901,Columbia University School of General Studies
1,http://www.wikidata.org/entity/Q4952707,Boys and Girls High School
2,http://www.wikidata.org/entity/Q5149833,Fu Foundation School of Engineering and Applie...
3,http://www.wikidata.org/entity/Q49088,Columbia University


__Fecha de las consultas: 28, 29, 30 y 31 de abril de 2022__